In [2]:
import duckdb

In [3]:
import pandas as pd
import numpy as np

# Problem 1 

In [4]:
# --- Orders Data ---
orders_data = {
    'OrderID': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'CustomerID': ['C1', 'C2', 'C1', 'C3', 'C2', 'C1', 'C4', 'C4', 'C2', 'C3'],
    'ProductID': [101, 102, 103, 101, 104, 102, 105, 101, 103, 105],
    'Quantity': [2, 1, 5, 3, 2, 1, 1, 1, 3, 2],
    'OrderDate': pd.to_datetime(['2025-04-01', '2025-04-01', '2025-04-02', '2025-04-02', '2025-04-03', '2025-04-03', '2025-04-04', '2025-04-05', '2025-04-05', '2025-04-06'])
}
orders_df = pd.DataFrame(orders_data)

In [5]:
# --- Products Data ---
# (This is the second part of the data you'll need)
products_data = {
    'ProductID': [101, 102, 103, 104, 105, 106],
    'ProductName': ['laptop', 'mouse', 'KEYBOARD', 'webcam', 'monitor', 'usb hub'],
    'Category': ['Electronics', 'Electronics', 'Electronics', 'Peripherals', 'Peripherals', 'Peripherals'],
    'UnitPrice': [1200, 25, 75, 50, -300, 20] # Note the negative price
}
products_df = pd.DataFrame(products_data)

In [6]:
# --- DuckDB Setup ---
# Connect to an in-memory database
conn = duckdb.connect(database=':memory:', read_only=False)

In [7]:
# Register both DataFrames as SQL tables
conn.register('orders', orders_df)
conn.register('products', products_df)

# print("Tables 'orders' and 'products' are registered with DuckDB.")
# print("You can now query them using conn.sql('SELECT * FROM orders')")

In [8]:

conn.execute("SELECT * FROM products").df()


,ProductID,ProductName,Category,UnitPrice
0,101,laptop,Electronics,1200
1,102,mouse,Electronics,25
2,103,KEYBOARD,Electronics,75
3,104,webcam,Peripherals,50
4,105,monitor,Peripherals,-300
5,106,usb hub,Peripherals,20


In [9]:

conn.execute("SELECT * FROM orders").df()


,OrderID,CustomerID,ProductID,Quantity,OrderDate
0,1,C1,101,2,2025-04-01
1,2,C2,102,1,2025-04-01
2,3,C1,103,5,2025-04-02
3,4,C3,101,3,2025-04-02
4,5,C2,104,2,2025-04-03
5,6,C1,102,1,2025-04-03
6,7,C4,105,1,2025-04-04
7,8,C4,101,1,2025-04-05
8,9,C2,103,3,2025-04-05
9,10,C3,105,2,2025-04-06


#### 1. Data Preparation & Merging 🔗
Combine the orders_df and products_df into a single DataFrame named sales_df. The final DataFrame should only include products that appear in the orders data.

The ProductName column has inconsistent capitalization. Standardize all product names to be in title case (e.g., 'laptop' should become 'Laptop').

In [10]:

conn.execute("""CREATE TABLE Sales AS
            SELECT o.* 
             ,p.ProductName
             ,p.Category
             ,p.UnitPrice
             FROM orders AS o
             JOIN products AS p
             ON o.ProductID =p.ProductID """).df()


,Count
0,10


In [11]:

conn.execute("SELECT * FROM Sales").df()


,OrderID,CustomerID,ProductID,Quantity,OrderDate,ProductName,Category,UnitPrice
0,1,C1,101,2,2025-04-01,laptop,Electronics,1200
1,2,C2,102,1,2025-04-01,mouse,Electronics,25
2,3,C1,103,5,2025-04-02,KEYBOARD,Electronics,75
3,4,C3,101,3,2025-04-02,laptop,Electronics,1200
4,5,C2,104,2,2025-04-03,webcam,Peripherals,50
5,6,C1,102,1,2025-04-03,mouse,Electronics,25
6,7,C4,105,1,2025-04-04,monitor,Peripherals,-300
7,8,C4,101,1,2025-04-05,laptop,Electronics,1200
8,9,C2,103,3,2025-04-05,KEYBOARD,Electronics,75
9,10,C3,105,2,2025-04-06,monitor,Peripherals,-300


In [12]:

conn.execute("""UPDATE Sales
             SET ProductName = Upper(substr(lower(ProductName),1,1)) 
             || 
             substr(lower(ProductName),2)""").df()


,Count
0,10



#### 2. Feature Engineering 🛠️
Create a new column named TotalOrderPrice. This should be the result of multiplying the Quantity by the UnitPrice. Note that this value will be negative for refunded items.

In [13]:

conn.execute("""ALTER TABLE Sales
        ADD COLUMN TotalOrderPrice DOUBLE
""").df()


,Success


In [14]:
conn.execute("""UPDATE Sales
             SET TotalOrderPrice = Quantity * UnitPrice
""").df()


,Count
0,10


In [15]:
conn.execute("""SELECT * FROM Sales
""").df()

,OrderID,CustomerID,ProductID,Quantity,OrderDate,ProductName,Category,UnitPrice,TotalOrderPrice
0,1,C1,101,2,2025-04-01,Laptop,Electronics,1200,2400.0
1,2,C2,102,1,2025-04-01,Mouse,Electronics,25,25.0
2,3,C1,103,5,2025-04-02,Keyboard,Electronics,75,375.0
3,4,C3,101,3,2025-04-02,Laptop,Electronics,1200,3600.0
4,5,C2,104,2,2025-04-03,Webcam,Peripherals,50,100.0
5,6,C1,102,1,2025-04-03,Mouse,Electronics,25,25.0
6,7,C4,105,1,2025-04-04,Monitor,Peripherals,-300,-300.0
7,8,C4,101,1,2025-04-05,Laptop,Electronics,1200,1200.0
8,9,C2,103,3,2025-04-05,Keyboard,Electronics,75,225.0
9,10,C3,105,2,2025-04-06,Monitor,Peripherals,-300,-600.0


#### 3. Data Analysis & Reporting 📊

Top Selling Products: Find the top 3 products that have generated the most revenue (i.e., have the highest total sum of TotalOrderPrice).

In [16]:
conn.execute(""" SELECT  ProductName , SUM(TotalOrderPrice) AS Revenue
             FROM Sales
             GROUP BY ProductName
             ORDER BY Revenue DESC
            LIMIT 3
""").df()

,ProductName,Revenue
0,Laptop,7200.0
1,Keyboard,600.0
2,Webcam,100.0


Most Expensive Item per Category: For each product category, find the name of the product with the highest UnitPrice. (For this task, you should ignore refunded items, so only consider products with a positive UnitPrice).

In [19]:
conn.execute("""SELECT Category, ProductName,UnitPrice
                FROM (
             SELECT * , 
             ROW_NUMBER() OVER(
             PARTITION BY Category
             ORDER BY UnitPrice DESC)
             AS rn
             FROM Sales
             WHERE UnitPrice > 0
             ) 
             WHERE rn = 1
""").df()

,Category,ProductName,UnitPrice
0,Peripherals,Webcam,50
1,Electronics,Laptop,1200


Customer Analysis: Identify any CustomerIDs who have a negative or zero total spending sum across all their orders. These could be customers who only returned items or whose refunds equal their purchases.

In [22]:
conn.execute("""
            SELECT CustomerID ,SUM(TotalOrderPrice) AS TotalSpent
             FROM Sales
                GROUP BY CustomerID
             HAVING SUM(TotalOrderPrice) <= 0
             
             """).df()

,CustomerID,TotalSpent


In [20]:
conn.execute("""
SELECT * FROM Sales
""").df()

,OrderID,CustomerID,ProductID,Quantity,OrderDate,ProductName,Category,UnitPrice,TotalOrderPrice
0,1,C1,101,2,2025-04-01,Laptop,Electronics,1200,2400.0
1,2,C2,102,1,2025-04-01,Mouse,Electronics,25,25.0
2,3,C1,103,5,2025-04-02,Keyboard,Electronics,75,375.0
3,4,C3,101,3,2025-04-02,Laptop,Electronics,1200,3600.0
4,5,C2,104,2,2025-04-03,Webcam,Peripherals,50,100.0
5,6,C1,102,1,2025-04-03,Mouse,Electronics,25,25.0
6,7,C4,105,1,2025-04-04,Monitor,Peripherals,-300,-300.0
7,8,C4,101,1,2025-04-05,Laptop,Electronics,1200,1200.0
8,9,C2,103,3,2025-04-05,Keyboard,Electronics,75,225.0
9,10,C3,105,2,2025-04-06,Monitor,Peripherals,-300,-600.0


# 2. Stock Market Time Series Analysis 📈

Stock Market Time Series Analysis 📈

In [23]:
import duckdb
import pandas as pd

# ---------------------------
# 1. Create the stocks_df DataFrame
# ---------------------------
data = {
    'Date': pd.to_datetime([
        '2025-09-01', '2025-09-02', '2025-09-03', '2025-09-04', '2025-09-05',
        '2025-09-01', '2025-09-02', '2025-09-03', '2025-09-04', '2025-09-05',
        '2025-09-08', '2025-09-09', '2025-09-10', '2025-09-08', '2025-09-09'
    ]),
    'Ticker': [
        'AAPL', 'AAPL', 'AAPL', 'AAPL', 'AAPL',
        'GOOG', 'GOOG', 'GOOG', 'GOOG', 'GOOG',
        'AAPL', 'AAPL', 'AAPL', 'GOOG', 'GOOG'
    ],
    'Close': [150, 152, 151, 155, 157, 2800, 2810, 2815, 2790, 2785, 156, 159, 160, 2795, 2805],
    'Volume': [1.2, 1.3, 1.1, 1.5, 1.6, 0.8, 0.9, 1.0, 1.2, 1.1, 1.4, 1.7, 1.5, 1.3, 1.4]
}

stocks_df = pd.DataFrame(data)

# Convert volume from millions to actual numbers
stocks_df['Volume'] = stocks_df['Volume'] * 1_000_000


# ---------------------------
# 2. Connect to DuckDB
# ---------------------------
# :memory: creates a temporary in-RAM database
conn = duckdb.connect(database=':memory:', read_only=False)


# ---------------------------
# 3. Register the pandas DataFrame
# ---------------------------
# This makes the DataFrame available inside DuckDB as "stocks_df"
conn.register('stocks_df', stocks_df)


# ---------------------------
# 4. Create a permanent SQL table inside DuckDB
# ---------------------------
conn.execute("""
    CREATE TABLE stocks AS 
    SELECT * FROM stocks_df
""")

print("Table 'stocks' successfully created in DuckDB!")


Table 'stocks' successfully created in DuckDB!


In [ ]:
conn.execute("""SELECT * FROM stocks

""").df()

This gives you a DataFrame with daily closing prices and trading volumes for Apple ('AAPL') and Google ('GOOG').

#### 1\. Data Preparation

  * To make time series operations easier, **set the `Date` column as the index** of the DataFrame.
  * The data is not in chronological order. **Sort the DataFrame by its index**.


In [25]:
conn.execute("""SELECT * FROM stocks
             ORDER BY Date ASC

""").df()

,Date,Ticker,Close,Volume
0,2025-09-01,AAPL,150,1200000.0
1,2025-09-01,GOOG,2800,800000.0
2,2025-09-02,AAPL,152,1300000.0
3,2025-09-02,GOOG,2810,900000.0
4,2025-09-03,AAPL,151,1100000.0
5,2025-09-03,GOOG,2815,1000000.0
6,2025-09-04,AAPL,155,1500000.0
7,2025-09-04,GOOG,2790,1200000.0
8,2025-09-05,AAPL,157,1600000.0
9,2025-09-05,GOOG,2785,1100000.0


#### 2\. Technical Indicators

  * Calculate the **3-day moving average** of the `Close` price for each stock. A moving average helps smooth out price fluctuations. The result should be a new column named `MA_3_Day`. (Note: The first two days for each stock will have `NaN` values, which is expected).


In [ ]:
conn.execute("""

""").df()

           Ticker  Close     Volume     MA_3_Day
Date                                            
2025-09-01   AAPL    150  1200000.0          NaN
2025-09-01   GOOG   2800   800000.0          NaN
2025-09-02   AAPL    152  1300000.0          NaN
2025-09-02   GOOG   2810   900000.0          NaN
2025-09-03   AAPL    151  1100000.0   151.000000
2025-09-03   GOOG   2815  1000000.0  2808.333333
2025-09-04   AAPL    155  1500000.0   152.666667
2025-09-04   GOOG   2790  1200000.0  2805.000000
2025-09-05   AAPL    157  1600000.0   154.333333
2025-09-05   GOOG   2785  1100000.0  2796.666667
2025-09-08   AAPL    156  1400000.0   156.000000
2025-09-08   GOOG   2795  1300000.0  2790.000000
2025-09-09   AAPL    159  1700000.0   157.333333
2025-09-09   GOOG   2805  1400000.0  2795.000000
2025-09-10   AAPL    160  1500000.0   158.333333


For each stock, take the last 3 closing prices, average them, and put that number in a new column. If there aren’t 3 days yet, put NaN

In [ ]:
conn.execute("""

""").df()

,Ticker,Close,Volume,MA_3_Day
Date,,,,
2025-09-01,AAPL,150,1200000.0,NaN
2025-09-01,GOOG,2800,800000.0,NaN
2025-09-02,AAPL,152,1300000.0,NaN
2025-09-02,GOOG,2810,900000.0,NaN
2025-09-03,AAPL,151,1100000.0,151.000000
2025-09-03,GOOG,2815,1000000.0,2808.333333
2025-09-04,AAPL,155,1500000.0,152.666667
2025-09-04,GOOG,2790,1200000.0,2805.000000
2025-09-05,AAPL,157,1600000.0,154.333333


#### 3\. Time Series Analysis & Reporting

  * **Resample the data to find the average weekly `Close` price** for each stock.
  * For each stock (`Ticker`), find the date on which it had its **highest trading `Volume`**.
  * Calculate the total percentage change in price for each stock from its first recorded date to its last. The formula is `(last_price - first_price) / first_price * 100`.

In [ ]:
conn.execute("""

""").df()

Ticker  Date      
AAPL    2025-09-07     153.000000
        2025-09-14     158.333333
GOOG    2025-09-07    2800.000000
        2025-09-14    2800.000000
Name: Close, dtype: float64


resample() only works when your DataFrame index is a DatetimeIndex.

In [ ]:
conn.execute("""

""").df()

C:\Users\ATW\AppData\Local\Temp\ipykernel_6624\154317282.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.loc[x["Volume"].idxmax()])


Group by ticker →
In each group pick the highest-Volume row →
Return all selected rows in a clean table

In [ ]:
conn.execute("""

""").df()

        Date Ticker  Close     Volume     MA_3_Day
0 2025-09-09   AAPL    159  1700000.0   157.333333
1 2025-09-09   GOOG   2805  1400000.0  2795.000000


In [ ]:
conn.execute("""

""").df()

,first,last
Ticker,,
AAPL,150,160
GOOG,2800,2805


In [ ]:
conn.execute("""

""").df()

,first,last,pct_change
Ticker,,,
AAPL,150,160,6.666667
GOOG,2800,2805,0.178571


In [ ]:
conn.execute("""

""").df()

In [ ]:
conn.execute("""

""").df()

,Ticker,Close,Volume,MA_3_Day,pct_change
Date,,,,,
2025-09-01,AAPL,150,1200000.0,NaN,6.666667
2025-09-01,GOOG,2800,800000.0,NaN,0.178571
2025-09-02,AAPL,152,1300000.0,NaN,6.666667
2025-09-02,GOOG,2810,900000.0,NaN,0.178571
2025-09-03,AAPL,151,1100000.0,151.000000,6.666667
2025-09-03,GOOG,2815,1000000.0,2808.333333,0.178571
2025-09-04,AAPL,155,1500000.0,152.666667,6.666667
2025-09-04,GOOG,2790,1200000.0,2805.000000,0.178571
2025-09-05,AAPL,157,1600000.0,154.333333,6.666667


In [ ]:
conn.execute("""

""").df()

# MyDb 

In [25]:
con = duckdb.connect("mydb.duckdb")

In [26]:
con.execute('CREATE TABLE IF NOT EXISTS employees (id INT PRIMARY KEY ,name VARCHAR(50),salary INT)')

In [27]:
con.execute( """INSERT INTO employees VALUES 
(1,'Adam',25000),
(2 ,'Bob',30000),
(3,'Casey',40000)
""")

ConstraintException: Constraint Error: Duplicate key "id: 1" violates primary key constraint.

In [ ]:
df = con.execute("SELECT * FROM employees").df()
print(df)

   id   name  salary
0   1   Adam   25000
1   2    Bob   30000
2   3  Casey   40000


# College

In [2]:
con = duckdb.connect("college.duckdb")

In [3]:
con.execute("""CREATE TABLE IF NOT EXISTS students (
            rollno INT PRIMARY KEY,
            name VARCHAR(50),
            marks INT CHECK (marks >= 0 AND marks <= 100),
            grade VARCHAR(1),
            city VARCHAR(20)
            )
""")

In [4]:
con.execute(
    """INSERT INTO students VALUES
    (101,'Anil',78,'C','Pune'),
    (102,'Bala',85,'B','Mumbai'),
    (103,'Chetan',92,'A','Delhi'),
    (104,'Dinesh',67,'D','Delhi'),
    (105,'Esha',74,'C','Pune'),
    (106,'Farah',88,'B','Mumbai'),
    (107,'Gopal',95,'A','Delhi'),
    (108,'Hina',81,'B','Pune'),
    (109,'Ishaan',69,'D','Jaipur'),
    (110,'Jaya',73,'C','Jaipur')"""
)

In [5]:
df = con.execute("SELECT * FROM students").df()
print(df)

   rollno    name  marks grade    city
0     101    Anil     78     C    Pune
1     102    Bala     85     B  Mumbai
2     103  Chetan     92     A   Delhi
3     104  Dinesh     67     D   Delhi
4     105    Esha     74     C    Pune
5     106   Farah     88     B  Mumbai
6     107   Gopal     95     A   Delhi
7     108    Hina     81     B    Pune
8     109  Ishaan     69     D  Jaipur
9     110    Jaya     73     C  Jaipur


In [6]:
df = con.execute('SELECT name,marks FROM students').df()
print(df)

     name  marks
0    Anil     78
1    Bala     85
2  Chetan     92
3  Dinesh     67
4    Esha     74
5   Farah     88
6   Gopal     95
7    Hina     81
8  Ishaan     69
9    Jaya     73


find avg. marks in each city in ascending order

In [7]:
df = con.execute(
    """SELECT city 
    FROM students
    GROUP BY city 
    ORDER BY city ASC"""
).df()
print(df)

     city
0   Delhi
1  Jaipur
2  Mumbai
3    Pune


In [8]:
df = con.execute(
    """SELECT city , AVG(marks)
    as average_marks 
    FROM students
    GROUP BY city
    ORDER BY city ASC"""
).df()
print(df)

     city  average_marks
0   Delhi      84.666667
1  Jaipur      71.000000
2  Mumbai      86.500000
3    Pune      77.666667


delete all the student who scored less than 80

In [ ]:
con.execute(
    """DELETE FROM students
    WHERE marks <80 """
)

   Count
0      5


In [14]:
con.execute(
    """ALTER TABLE students
    DROP COLUMN grade"""
)

In [15]:
df = con.execute(
    'SELECT * FROM students'
).df()
print(df)

   rollno full_name  marks    city
0     102      Bala     85  Mumbai
1     103    Chetan     92   Delhi
2     106     Farah     88  Mumbai
3     107     Gopal     95   Delhi
4     108      Hina     81    Pune


In [ ]:
con.execute(
    """ALTER TABLE students
    CHANGE name TO full_name"""
)

# Company

In [28]:
# Connect (creates a DB file, or keep in memory with duckdb.connect())
con = duckdb.connect("company.duckdb")

In [29]:
# Step 1: Create the table
con.execute("""
CREATE TABLE IF NOT EXISTS customers (
    customer_id INT PRIMARY KEY,
    customer VARCHAR(50),
    mode VARCHAR(20),
    city VARCHAR(20)
)
""")

In [30]:
# Step 2: Insert rows
con.execute("""
INSERT INTO customers VALUES
(101, 'Olivia Barrett', 'Netbanking', 'Portland'),
(102, 'Ethan Sinclair', 'Credit Card', 'Miami'),
(103, 'Maya Hernandez', 'Credit Card', 'Seattle'),
(104, 'Liam Donovan', 'Netbanking', 'Denver'),
(105, 'Sophia Nguyen', 'Credit Card', 'New Orleans'),
(106, 'Caleb Foster', 'Debit Card', 'Minneapolis'),
(107, 'Ava Patel', 'Debit Card', 'Phoenix'),
(108, 'Lucas Carter', 'Netbanking', 'Boston'),
(109, 'Isabella Martinez', 'Netbanking', 'Nashville'),
(110, 'Jackson Brooks', 'Credit Card', 'Boston')
""")

In [31]:
# Step 3: View all data
df = con.execute("SELECT * FROM customers").df()
print(df)

   customer_id           customer         mode         city
0          101     Olivia Barrett   Netbanking     Portland
1          102     Ethan Sinclair  Credit Card        Miami
2          103     Maya Hernandez  Credit Card      Seattle
3          104       Liam Donovan   Netbanking       Denver
4          105      Sophia Nguyen  Credit Card  New Orleans
5          106       Caleb Foster   Debit Card  Minneapolis
6          107          Ava Patel   Debit Card      Phoenix
7          108       Lucas Carter   Netbanking       Boston
8          109  Isabella Martinez   Netbanking    Nashville
9          110     Jackson Brooks  Credit Card       Boston


for the given table , find the total payment according to each payment method

In [36]:
df = con.execute(
    """SELECT mode 
    , COUNT(mode) AS total_payment
    FROM customers
    GROUP BY mode"""
).df()
print(df)

          mode  total_payment
0   Debit Card              2
1  Credit Card              4
2   Netbanking              4


change  the name of column name to full_name

In [ ]:
con.execute(
    '''ALTER TABLE customers 
    RENAME COLUMN customer TO full_name''')

In [38]:
df = con.execute("SELECT * FROM customers").df()
print(df)

   customer_id          full_name         mode         city
0          101     Olivia Barrett   Netbanking     Portland
1          102     Ethan Sinclair  Credit Card        Miami
2          103     Maya Hernandez  Credit Card      Seattle
3          104       Liam Donovan   Netbanking       Denver
4          105      Sophia Nguyen  Credit Card  New Orleans
5          106       Caleb Foster   Debit Card  Minneapolis
6          107          Ava Patel   Debit Card      Phoenix
7          108       Lucas Carter   Netbanking       Boston
8          109  Isabella Martinez   Netbanking    Nashville
9          110     Jackson Brooks  Credit Card       Boston
